In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
import math
from math import log

from  pprint import pformat
from collections import namedtuple

In [3]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
           ['青年', '否', '否', '好', '否'],
           ['青年', '是', '否', '好', '是'],
           ['青年', '是', '是', '一般', '是'],
           ['青年', '否', '否', '一般', '否'],
           ['中年', '否', '否', '一般', '否'],
           ['中年', '否', '否', '好', '否'],
           ['中年', '是', '是', '好', '是'],
           ['中年', '否', '是', '非常好', '是'],
           ['中年', '否', '是', '非常好', '是'],
           ['老年', '否', '是', '非常好', '是'],
           ['老年', '否', '是', '好', '是'],
           ['老年', '是', '否', '好', '是'],
           ['老年', '是', '否', '非常好', '是'],
           ['老年', '否', '否', '一般', '否'],
           ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    
    return datasets,labels

In [4]:
datasets,labels=create_data()

In [5]:
train_data=pd.DataFrame(datasets,columns=labels)

In [6]:
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [7]:
train_data2=train_data
train_data2.columns=['Age','Work','Hourse','Loan','Category']

In [8]:
def emp_ent(col):
    data_length = len(col)
    label_count = {}
#     for i in range(data_length):
#         label = datasets[i][-1]
#         if label not in label_count:
#             label_count[label] = 0
#         label_count[label] += 1
    label_count=dict(col.value_counts())
    ent = -(sum((p/data_length) * math.log((p/data_length),2) for p in label_count.values()))
    return ent

In [9]:
emp_ent(train_data2.Category)

0.9709505944546686

In [10]:
def cond_ent(data,feat_col,label_col):
    lfeat=len(data[feat_col])
    feat_value=data[feat_col].unique().tolist()
    cent=0
    for fv in feat_value:
        eent=emp_ent(data[data[feat_col]==fv][label_col])
        p=len(data[data[feat_col]==fv][feat_col])/lfeat
        cent+=eent*p
    return cent
        
    

In [11]:
cond_ent(train_data2,'Age','Category')

0.8879430945988998

In [12]:
def gini_func(data,feat_col,label_col):
    return emp_ent(data[label_col])-cond_ent(data,feat_col,label_col)

In [13]:
gini_func(train_data2,'Age','Category')

0.08300749985576883

In [14]:
def find_max_gini(data,label_col):
    feat_name=data.columns.values.tolist()
    feat_name.remove(label_col)
    gini_result={}
    for fname in feat_name:
        gini_result[fname]=gini_func(data,fname,label_col)
    max_val=sorted(gini_result.items(),key=lambda x:x[1])[-1]
    print("max_gini : "+str(max_val))
    return max_val
        

In [15]:
find_max_gini(train_data2,'Category')

max_gini : ('Hourse', 0.4199730940219749)


('Hourse', 0.4199730940219749)

In [16]:
class tree(namedtuple('dt_tree','feat,value')):
    def __repr__(self):
        return pformat(tuple(self))

In [17]:
def dt_train(data,label_col):
    feat_name=data.columns.values.tolist()
    feat_name.remove(label_col)
    
    if len(feat_name)<=1:
        return tree(None,None)
    
    ## find max gini
    fname,gvalue=find_max_gini(data,label_col)
    print('fname:'+str(fname))
    
    feat_value=data[fname].unique().tolist()
    print('feat_name:'+str(feat_value))
    next_value=''
    for fvalue in feat_value:
        print('fvalue: '+str(fvalue))
        label_values=data[data[fname]==fvalue][label_col].unique().tolist()
        if len(label_values) ==1:
            lvalue=label_values
            next_value+='{'+str(fvalue)+':{label:'+str(lvalue)+'}}'
        else:
            feat_name1=feat_name
            feat_name1.remove(fname)
            feat_name1.append(label_col)
            temp=dt_train(data[data[fname]==fvalue][feat_name1],label_col)
            next_value+='{'+str(fvalue)+'{'+str(temp.feat)+','+str(temp.value)+'}}'
    return tree(fname,next_value)


In [18]:
dt_train(train_data2,'Category')

max_gini : ('Hourse', 0.4199730940219749)
fname:Hourse
feat_name:['否', '是']
fvalue: 否
max_gini : ('Work', 0.9182958340544896)
fname:Work
feat_name:['否', '是']
fvalue: 否
fvalue: 是
fvalue: 是


('Hourse', "{否{Work,{否:{label:['否']}}{是:{label:['是']}}}}{是:{label:['是']}}")

In [19]:
class tree2(namedtuple('dt_tree','feat,label,value,subtree')):
    def __repr__(self):
        return pformat(tuple(self))

In [20]:
def dt_train2(data,label_col):
    feat_name=data.columns.values.tolist()
    feat_name.remove(label_col)
    
    if len(feat_name)<=1:
        return tree(None,None,None,None)
    
    ## find max gini
    fname,gvalue=find_max_gini(data,label_col)
    print('fname:'+str(fname))
    
    feat_value=data[fname].unique().tolist()
    print('feat_name:'+str(feat_value))
#     next_value=''
    tl=''
    tv=''
    st=''
    for fvalue in feat_value:
        print('fvalue: '+str(fvalue))
        label_values=data[data[fname]==fvalue][label_col].unique().tolist()
        if len(label_values) ==1:
            lvalue=label_values
#             next_value+='{'+str(fvalue)+':{label:'+str(lvalue)+'}}'
            tl=label_values
            tv=fvalue
#             return tree2(fname,label_values,fvalue,None)
        else:
            feat_name1=feat_name
            feat_name1.remove(fname)
            feat_name1.append(label_col)
            temp=dt_train2(data[data[fname]==fvalue][feat_name1],label_col)
#             next_value+='{'+str(fvalue)+'{'+str(temp.feat)+','+str(temp.value)+'}}'
            st=tree2(fname,None,fvalue,temp) 
            return tree2(fname,None,fvalue,temp) 
            
    return tree2(fname,tl,tv,st)


In [21]:
print(dt_train2(train_data2,'Category'))

max_gini : ('Hourse', 0.4199730940219749)
fname:Hourse
feat_name:['否', '是']
fvalue: 否
max_gini : ('Work', 0.9182958340544896)
fname:Work
feat_name:['否', '是']
fvalue: 否
fvalue: 是
('Hourse', None, '否', ('Work', ['是'], '是', ''))


In [22]:
dt_train2(train_data2,'Category')

max_gini : ('Hourse', 0.4199730940219749)
fname:Hourse
feat_name:['否', '是']
fvalue: 否
max_gini : ('Work', 0.9182958340544896)
fname:Work
feat_name:['否', '是']
fvalue: 否
fvalue: 是


('Hourse', None, '否', ('Work', ['是'], '是', ''))

In [23]:
class tree3(namedtuple('dt_tree','feat,value,label,subtree')):
    def __repr__(self):
        return pformat(tuple(self))

In [24]:
def dt_train3(data,label_col):
    feat_name=data.columns.values.tolist()
    feat_name.remove(label_col)
    
    if len(feat_name)<=1:
        return tree3(None,None,None,None)
    
    ## find max gini
    fname,gvalue=find_max_gini(data,label_col)
    print('fname:'+str(fname))
    
    feat_value=data[fname].unique().tolist()
    print('feat_name:'+str(feat_value))
    next_value=''
    lv=None
    fv=None
    nv=[]
    for fvalue in feat_value:
        print('fvalue: '+str(fvalue))
        label_values=data[data[fname]==fvalue][label_col].unique().tolist()
        if len(label_values) ==1:
            lv=label_values
            fv=fvalue
            # next_value+='{'+str(fvalue)+':{label:'+str(lvalue)+'}}'
#             next_value+=',{'+str(fvalue)+str(lvalue)+'}'
            nv.append(tree3(None,fv,lv,None))
            
            
        else:
            feat_name1=feat_name
            feat_name1.remove(fname)
            feat_name1.append(label_col)
            temp=dt_train3(data[data[fname]==fvalue][feat_name1],label_col)
#             next_value+='{'+str(fvalue)+'{'+str(temp.feat)+','+str(temp.value)+'}}'
            fv=fvalue
            nv.append(tree3(None,fv,None,temp))
    return tree3(fname,None,None,nv)


In [25]:
dt_train3(train_data2,'Category')

max_gini : ('Hourse', 0.4199730940219749)
fname:Hourse
feat_name:['否', '是']
fvalue: 否
max_gini : ('Work', 0.9182958340544896)
fname:Work
feat_name:['否', '是']
fvalue: 否
fvalue: 是
fvalue: 是


('Hourse',
 None,
 None,
 [(None,
 '否',
 None,
 ('Work', None, None, [(None, '否', ['否'], None), (None, '是', ['是'], None)])),
  (None, '是', ['是'], None)])

In [26]:
from time import clock

In [27]:
t0=clock()
dt3=dt_train3(train_data2,'Category')
t1=clock()
print(t1-t0)

max_gini : ('Hourse', 0.4199730940219749)
fname:Hourse
feat_name:['否', '是']
fvalue: 否
max_gini : ('Work', 0.9182958340544896)
fname:Work
feat_name:['否', '是']
fvalue: 否
fvalue: 是
fvalue: 是
0.03480066917904272


In [28]:
t_data=np.array(['老年', '否', '否', '一般'])
t_data=t_data.reshape(1,4)
t_col=train_data2.columns.values.tolist()[:-1]
# test_data=pd.DataFrame(np.array(t_data),columns=t_col)
test_data=pd.DataFrame(t_data,columns=t_col)
test_data

,Age,Work,Hourse,Loan
0,老年,否,否,一般


In [29]:
dt3.subtree[0].value

'否'

In [30]:
if dt3.subtree[0].feat is None:
    print('Not')

Not


In [31]:
dt3.subtree

[(None,
 '否',
 None,
 ('Work', None, None, [(None, '否', ['否'], None), (None, '是', ['是'], None)])),
 (None, '是', ['是'], None)]

In [32]:
dt3.subtree[0].subtree

('Work', None, None, [(None, '否', ['否'], None), (None, '是', ['是'], None)])

In [33]:
dt3.subtree[0].subtree

('Work', None, None, [(None, '否', ['否'], None), (None, '是', ['是'], None)])

In [34]:
dt3.subtree[0].value=='否'

True

In [35]:
def dt3_predict(trained_tree,test_data,last_feat=None):
    print('-----------')
    print(trained_tree)
    print('-----------')
    f=trained_tree.feat
    if f is None:
        f=last_feat
    l=trained_tree.label
    v=trained_tree.value
    tv=test_data[f].values[0]
    print('l:'+str(l))
    print('v:'+str(v))
    print('f:'+str(f))
    print('tv:'+str(tv))
    if ((tv==v) and (l is not None) ):
        return l

    if tv==v:
        print('tv == v')
        l=dt3_predict(trained_tree.subtree,test_data,f)


    
    if l is  None:
        print( 'tv ! = v')
        i=0
        while trained_tree.subtree[i].value != tv  :
            i+=1
        print('i:'+str(i))
        print('---'+str(trained_tree.subtree[i].value))
        print('---'+str(trained_tree.subtree[i]))
        l=dt3_predict(trained_tree.subtree[i],test_data,f)
    return l[0]
    

In [36]:
from time import clock

In [37]:
t0=clock()
dt3_predict(dt3,test_data)
t1=clock()
print(t1-t0)

-----------
('Hourse',
 None,
 None,
 [(None,
 '否',
 None,
 ('Work', None, None, [(None, '否', ['否'], None), (None, '是', ['是'], None)])),
  (None, '是', ['是'], None)])
-----------
l:None
v:None
f:Hourse
tv:否
tv ! = v
i:0
---否
---(None,
 '否',
 None,
 ('Work', None, None, [(None, '否', ['否'], None), (None, '是', ['是'], None)]))
-----------
(None,
 '否',
 None,
 ('Work', None, None, [(None, '否', ['否'], None), (None, '是', ['是'], None)]))
-----------
l:None
v:否
f:Hourse
tv:否
tv == v
-----------
('Work', None, None, [(None, '否', ['否'], None), (None, '是', ['是'], None)])
-----------
l:None
v:None
f:Work
tv:否
tv ! = v
i:0
---否
---(None, '否', ['否'], None)
-----------
(None, '否', ['否'], None)
-----------
l:['否']
v:否
f:Work
tv:否
0.0033269233221461247


In [38]:
l=None
tv=1
v=1
if tv==v and l is None:
    print('tttttt')


tttttt


In [39]:
test_data.loc[:,'Hourse']

0    否
Name: Hourse, dtype: object

sklearn DT

In [40]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [41]:
# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, [0, 1, -1]])
    # print(data)
    return data[:,:2], data[:,-1]

X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [43]:
clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [44]:
clf.score(X_test,y_test)

1.0

In [46]:
from sklearn.tree import export_graphviz
import graphviz

In [47]:
tree_pic=export_graphviz(clf,out_file='mytree1.pdf')

In [53]:
with open('mytree1.pdf','r') as f:
#     print(f.read())
    dot_graph=f.read()

In [57]:
# graphviz.Source(dot_graph)

In [56]:
%run Decision_Tree_Cut_Page

DTree:
0 (2, None)
1 (1, '否')
2 (0, '否')
3 (3, '青年')
4 (None, '一般', '否', 3)
4 (None, '好', '否', 1)
3 (None, '中年', '否', 2)
3 (None, '老年', '否', 1)
2 (None, '是', '是', 3)
1 (None, '是', '是', 6)
result: 否
Ent: [0.9182958340544896, 0.0, 0.0, 0.0, 0.0, 0.0]
C(T): 5.754887502163468
Ent: [0.8112781244591328, 0.0, 0.0, 0.0, 0.0]
C(T): 5.745112497836532
Ent: [0.5916727785823275, 0.0, 0.0]
C(T): 5.641709450076292
Ent: [0.9709505944546686, 0.0]
C(T): 10.709505944546686
DTree:
0 (2, None)
1 (1, '否')
2 (None, '否', '否', 7)
2 (None, '是', '是', 3)
1 (None, '是', '是', 6)
result: 否
